In [42]:
import json
from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

In [5]:
city = 'bogota'
locator_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})
locator_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=bogota&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [6]:
result = requests.get(locator_url).json()
result

{'response': {'results': {'results': [{'id': '3688689',
     'name': 'Bogotá',
     'container': 'Colombia',
     'containerId': 3686110,
     'language': 'en',
     'timezone': 'America/Bogota',
     'country': 'CO',
     'latitude': 4.60971,
     'longitude': -74.08175,
     'placeType': 'settlement'}],
   'totalResults': 1}}}

In [7]:
location_id = result['response']['results']['results'][0]['id']
location_id

'3688689'

In [31]:
def get_loc_id (enter_city):
  city1 = enter_city.lower()
  url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': city1,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})
  result = requests.get(url).json()
  return result ['response']['results']['results'][0]['id']

In [32]:
get_loc_id('Dehra dun')

'1273313'

In [34]:
url = 'https://www.bbc.com/weather/'+result['response']['results']['results'][0]['id']
response = requests.get(url)

In [67]:
url = 'https://www.bbc.com/weather/'+get_loc_id('Bogota')
response = requests.get(url)

In [35]:
response

<Response [200]>

In [68]:
soup = BeautifulSoup(response.content,'html.parser')

In [69]:
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary

<div class="wr-day-summary"><div class="gel-wrap"><span class="">Light rain showers and a gentle breeze</span><span class="wr-hide">Light rain showers and a gentle breeze</span><span class="wr-hide">Light rain showers and a gentle breeze</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Thundery showers and light winds</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Light rain showers and light winds</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Thundery showers and light winds</span><span class="wr-hide">Thundery showers and light winds</span></div></div>

In [70]:
daily_summary.text

'Light rain showers and a gentle breezeLight rain showers and a gentle breezeLight rain showers and a gentle breezeLight rain showers and light windsThundery showers and light windsLight rain showers and light windsLight rain showers and light windsLight rain showers and light windsLight rain showers and light windsLight rain showers and light windsThundery showers and a gentle breezeThundery showers and a gentle breezeThundery showers and light windsThundery showers and light winds'

In [71]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase
daily_summary_list

['Light rain showers and a gentle breeze',
 'Light rain showers and a gentle breeze',
 'Light rain showers and a gentle breeze',
 'Light rain showers and light winds',
 'Thundery showers and light winds',
 'Light rain showers and light winds',
 'Light rain showers and light winds',
 'Light rain showers and light winds',
 'Light rain showers and light winds',
 'Light rain showers and light winds',
 'Thundery showers and a gentle breeze',
 'Thundery showers and a gentle breeze',
 'Thundery showers and light winds',
 'Thundery showers and light winds']

In [44]:
datelist = pd.date_range(datetime.today(), periods=len(daily_summary_list)).tolist()
datelist

[Timestamp('2025-02-05 17:50:07.967860'),
 Timestamp('2025-02-06 17:50:07.967860'),
 Timestamp('2025-02-07 17:50:07.967860'),
 Timestamp('2025-02-08 17:50:07.967860'),
 Timestamp('2025-02-09 17:50:07.967860'),
 Timestamp('2025-02-10 17:50:07.967860'),
 Timestamp('2025-02-11 17:50:07.967860'),
 Timestamp('2025-02-12 17:50:07.967860'),
 Timestamp('2025-02-13 17:50:07.967860'),
 Timestamp('2025-02-14 17:50:07.967860'),
 Timestamp('2025-02-15 17:50:07.967860'),
 Timestamp('2025-02-16 17:50:07.967860'),
 Timestamp('2025-02-17 17:50:07.967860'),
 Timestamp('2025-02-18 17:50:07.967860')]

In [45]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
datelist

['25-02-05',
 '25-02-06',
 '25-02-07',
 '25-02-08',
 '25-02-09',
 '25-02-10',
 '25-02-11',
 '25-02-12',
 '25-02-13',
 '25-02-14',
 '25-02-15',
 '25-02-16',
 '25-02-17',
 '25-02-18']

In [46]:
zipped = zip(datelist, daily_summary_list)

In [47]:
df = pd.DataFrame(list(zipped), columns=['Date','Summary'])
df

,Date,Summary
0,25-02-05,Light rain showers and a gentle breeze
1,25-02-06,Light rain showers and a gentle breeze
2,25-02-07,Light rain showers and a gentle breeze
3,25-02-08,Light rain showers and light winds
4,25-02-09,Thundery showers and light winds
5,25-02-10,Light rain showers and light winds
6,25-02-11,Light rain showers and light winds
7,25-02-12,Light rain showers and light winds
8,25-02-13,Light rain showers and light winds
9,25-02-14,Light rain showers and light winds


In [48]:
location = soup.find('h1', attrs={'id':'wr-location-name-id'})
location.text.split()

['Bogotá', '-', 'Weather', 'warnings', 'issued']

In [52]:
json_data = df.to_json(orient='values')
json_data

'[["25-02-05","Light rain showers and a gentle breeze"],["25-02-06","Light rain showers and a gentle breeze"],["25-02-07","Light rain showers and a gentle breeze"],["25-02-08","Light rain showers and light winds"],["25-02-09","Thundery showers and light winds"],["25-02-10","Light rain showers and light winds"],["25-02-11","Light rain showers and light winds"],["25-02-12","Light rain showers and light winds"],["25-02-13","Light rain showers and light winds"],["25-02-14","Light rain showers and light winds"],["25-02-15","Thundery showers and a gentle breeze"],["25-02-16","Thundery showers and a gentle breeze"],["25-02-17","Thundery showers and light winds"],["25-02-18","Thundery showers and light winds"]]'